<a href="https://colab.research.google.com/github/sainikhit2k/100-Day-of-DL/blob/master/DCGAN/DCGAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, UpSampling2D, Activation, Dropout
from keras.layers import Input, Dense, BatchNormalization, Reshape, Flatten
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import LeakyReLU


# Uitlity libraries
import matplotlib.pyplot as plt
import numpy as np

# Libraries for data
from keras.datasets import fashion_mnist

Using TensorFlow backend.


In [2]:
print(keras.__version__)

2.3.1


In [0]:
img_width = 28
img_height = 28
num_channels = 1
latent_dim = 100
img_shape = (img_height, img_width, num_channels)
batch_size = 128
epochs = 4000
save_interval = 50

In [0]:
def build_generator():
  model = Sequential()
  model.add(Dense(128 * 7 * 7, activation="relu", input_shape=(latent_dim,)))
  model.add(Reshape((7,7,128)))
  model.add(UpSampling2D()) # shape = (14, 14, 128)
  model.add(Conv2D(128, kernel_size=3, padding="same")) # shape = (14, 14, 128)
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D()) # shape = (28, 28, 128)
  model.add(Conv2D(64, kernel_size=3, padding="same")) # shape = (28, 28, 64)
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(Conv2D(32, kernel_size=3, padding="same")) # shape = (28, 28, 32)
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(Conv2D(num_channels,kernel_size=3, padding="same")) # shape = (28, 28, 1)
  model.add(Activation("tanh"))
  
  # model.summary()

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(noise, img)



In [0]:
def build_discriminator():
  model = Sequential()
  model.add(Conv2D(256, kernel_size=3, padding="same", input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(128, kernel_size=3, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(64, kernel_size=3, padding="valid"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(32, kernel_size=3, padding="valid"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Flatten())
  model.add(Dense(1, activation="sigmoid"))

  # model.summary()
  
  imgs = Input(shape=img_shape)
  pred = model(imgs)

  return Model(imgs, pred)

In [0]:
import os

def save_img(epoch, generator):
  if os.path.isdir("/images") == False:
    os.mkdir("/images")

  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  # fig.savefig("/content/drive/My Drive/DCGAN/images/mnist_%d.png" % epoch)
  plt.close()

In [0]:
def train_DCGAN():
  (X_train, _), (_, _) = fashion_mnist.load_data()
  X_train = X_train / 127.5 - 1.
  X_train = np.expand_dims(X_train, axis=3)

  # print(type(X_train))

  discriminator = build_discriminator()
  discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])

  generator = build_generator()
  generator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))

  D_loss = []
  G_loss = []

  z = Input(shape=(latent_dim,))
  img = generator(z)
  discriminator.trainable = False
  pred = discriminator(img)
  combined = Model(z, pred)
  combined.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])

  valid = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for epoch in range(epochs):

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    
    imgs = X_train[idx]
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)


    g_loss = combined.train_on_batch(noise, valid)

    D_loss.append(d_loss[0])
    G_loss.append(g_loss)

    print(f'Epoch {epoch}, [D_Loss: {d_loss[0]}, acc: {100*d_loss[1]}% ], G_loss: {g_loss[0]}')
    
    if epoch % save_interval == 0:
      save_img(epoch, generator)
  
  return D_loss, G_loss



In [14]:
d_loss, g_loss = train_DCGAN()

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 0, [D_Loss: 1.2378009557724, acc: 25.0% ], G_loss: 0.6732935905456543
Epoch 1, [D_Loss: 0.5767718553543091, acc: 65.234375% ], G_loss: 1.2355890274047852
Epoch 2, [D_Loss: 0.13882218301296234, acc: 100.0% ], G_loss: 1.5067412853240967
Epoch 3, [D_Loss: 0.07947030663490295, acc: 99.609375% ], G_loss: 1.3042590618133545
Epoch 4, [D_Loss: 0.05746994912624359, acc: 99.609375% ], G_loss: 1.103917121887207
Epoch 5, [D_Loss: 0.033509816974401474, acc: 100.0% ], G_loss: 0.7751242518424988
Epoch 6, [D_Loss: 0.03327786922454834, acc: 100.0% ], G_loss: 0.6629794836044312
Epoch 7, [D_Loss: 0.03270099684596062, acc: 100.0% ], G_loss: 0.6866230368614197
Epoch 8, [D_Loss: 0.03796464204788208, acc: 100.0% ], G_loss: 0.6861624717712402
Epoch 9, [D_Loss: 0.04412443935871124, acc: 100.0% ], G_loss: 0.9706522226333618
Epoch 10, [D_Loss: 0.04011695832014084, acc: 100.0% ], G_loss: 1.3775980472564697
Epoch 11, [D_Loss: 0.057765595614910126, acc: 99.609375% ], G_loss: 2.1936280727386475
Epoch 12, [D_Lo

NameError: ignored